# Запуск собственных функций

При составлении цепочек вы можете использовать собственные функции.

При этом все входные данные для таких функций должны быть представлены одним аргументом.
Если ваша функция принимает несколько аргументов, для нее нужно написать обертку, которая принимает на входе один аргумент и распаковывает его в несколько аргументов.

In [1]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chat_models.gigachat import GigaChat


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = GigaChat(credentials="NjgxZmQ1Y2ItMWVmNS00Y2ZmLWE2MDgtZmZjNDM0NDA3NmJlOjA2NTNhNjAwLTU5NmYtNGQ0Ni05N2MxLTVhYjI0NWU0OTBlOA==", verify_ssl_certs=False, scope="GIGACHAT_API_CORP")

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [2]:
chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 = 12', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd204541-81fd-429a-ad92-dd1913af9b1c-0')

## Конфигурирование Runnable

Runnable-лямбды могут принимать в объекты [`RunnableConfig`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.config.RunnableConfig.html#langchain_core.runnables.config.RunnableConfig), которые используются для передачи обратных вызовов, тегов и других параметров вложенных исполняемых компонентнов.

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig

In [4]:
import json


def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | GigaChat(credentials="NjgxZmQ1Y2ItMWVmNS00Y2ZmLWE2MDgtZmZjNDM0NDA3NmJlOjA2NTNhNjAwLTU5NmYtNGQ0Ni05N2MxLTVhYjI0NWU0OTBlOA==", verify_ssl_certs=False, scope="GIGACHAT_API_CORP")
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

In [5]:
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 62
	Prompt Tokens: 56
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $9.6e-05


## Потоковая передача токенов

В LCEL-конвейере можно использовать функции-генераторы: работающие как итерароры функции, которые использут ключевое слово `yield`.

Сигнатура таких функций должна быть `Iterator[Input] -> Iterator[Output]` или `AsyncIterator[Input] -> AsyncIterator[Output]` при работе в асинхронном режиме.

Используйте функции-генераторы, когда:

* добавляете собственный парсер выходных данных;
* нужно сохранить возможность работать в режиме потоковой передачи токенов при преобразовании выходные данных, полученны на последнем шаге.

Ниже представлены примеры синхронной и асинхронной версии собственного парсера выходных данных для разделенных запятой перечислений.

### Синхронная версия

In [6]:
from typing import Iterator, List

prompt = ChatPromptTemplate.from_template(
    "Write a comma-separated list of 5 animals similar to: {animal}. Do not include numbers"
)
model = ChatOpenAI(temperature=0.0)

str_chain = prompt | model | StrOutputParser()

In [7]:
for chunk in str_chain.stream({"animal": "bear"}):
    print(chunk, end="", flush=True)

lion, tiger, wolf, gorilla, panda

In [8]:
str_chain.invoke({"animal": "bear"})

'lion, tiger, wolf, gorilla, panda'

In [9]:
# Собственный парсер, который делит итератор llm-токенов
# на список строк, разделенных запятыми
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    # сохранение части ввода до получения запятой
    buffer = ""
    for chunk in input:
        # добавление текущего фрагмента в буфер
        buffer += chunk
        # пока в буфере есть запятые
        while "," in buffer:
            # деление буфера при обнаружении запятой
            comma_index = buffer.index(",")
            # получение данных перед запятой
            yield [buffer[:comma_index].strip()]
            # сохранение оставшихся данных для следующей итерации
            buffer = buffer[comma_index + 1 :]
    # получение последнего фрагмента
    yield [buffer.strip()]

In [10]:
list_chain = str_chain | split_into_list

In [11]:
for chunk in list_chain.stream({"animal": "bear"}):
    print(chunk, flush=True)

['lion']
['tiger']
['wolf']
['gorilla']
['panda']


In [12]:
list_chain.invoke({"animal": "bear"})

['lion', 'tiger', 'wolf', 'gorilla', 'elephant']

### Асинхронная версия

In [13]:
from typing import AsyncIterator


async def asplit_into_list(
    input: AsyncIterator[str],
) -> AsyncIterator[List[str]]:  # async def
    buffer = ""
    async for (
        chunk
    ) in input:  # `input` — экземпляр `async_generator`, поэтому нужно использовать `async for`
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1 :]
    yield [buffer.strip()]


list_chain = str_chain | asplit_into_list

In [14]:
async for chunk in list_chain.astream({"animal": "bear"}):
    print(chunk, flush=True)

['lion']
['tiger']
['wolf']
['gorilla']
['panda']


In [15]:
await list_chain.ainvoke({"animal": "bear"})

['lion', 'tiger', 'wolf', 'gorilla', 'panda']